<a href="https://colab.research.google.com/github/issei/DaedalusForge/blob/main/automations/lan%C3%A7amento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geração de Copy para Lançamentos com Agentes de IA em Grafo

### Objetivo Principal
Este notebook automatiza a criação de textos de marketing (copy) para o lançamento de um infoproduto. Utilizando um briefing detalhado como ponto de partida, uma rede de agentes de IA, orquestrada com **LangGraph**, gera de forma colaborativa e iterativa os principais ativos de comunicação necessários para a campanha.

### Arquitetura
A solução é construída sobre uma arquitetura que combina um Large Language Model (LLM) com um framework de orquestração de grafos e uma base de conhecimento vetorial (RAG).

- **LLM**: **Google Gemini 1.5 Flash**, um modelo rápido e eficiente para geração de texto.
- **Framework de Orquestração**: **LangGraph**, para criar um fluxo de trabalho cíclico e com estado, permitindo que os agentes colaborem e refinem o trabalho uns dos outros.
- **RAG (Retrieval-Augmented Generation)**: **DumplingAI** atua como uma base de conhecimento vetorial. O briefing do lançamento é indexado e pode ser consultado pelos agentes para garantir que a copy gerada seja consistente e alinhada à estratégia.

### Fluxo de Execução com LangGraph
O processo é gerenciado por um grafo de estados que coordena os agentes:

1.  **Configuração e Indexação**: O ambiente é preparado, e o briefing do lançamento é carregado e indexado na base de conhecimento (RAG).
2.  **Análise Inicial (Paralela)**: Três agentes especializados (`Dores & Promessas`, `Objeções & Quebras`, `Headlines & Ângulos`) analisam o briefing simultaneamente para extrair os insights fundamentais.
3.  **Consolidação de Contexto**: Um nó `Consolidador` reúne as análises iniciais, criando um "super contexto" enriquecido que servirá de base para a criação da copy.
4.  **Geração da Copy**: O agente de `Adaptação por Canais` utiliza o contexto enriquecido para criar as primeiras versões da copy para as diferentes plataformas (Email, Ads, etc.).
5.  **Ciclo de Revisão e Refinamento**:
    - Um agente `Crítico Revisor` avalia a copy gerada, comparando-a com o briefing original.
    - Se a copy for **"APROVADA"**, o fluxo termina.
    - Se for marcada para **"REFINAR"**, o agente de `Adaptação` recebe o feedback e gera uma nova versão, iniciando um novo ciclo de revisão.
6.  **Saída Final**: Após a aprovação, a versão final da copy é salva em arquivos `JSON` e `Markdown`, pronta para uso.

In [1]:
!pip -q install -U langchain langchain-google-genai google-generativeai httpx pydantic python-dotenv langgraph
print("✅ Dependências instaladas")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.9/444.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 995.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 10.9 MB/s eta 0:00:00
✅ Dependências instaladas


In [2]:
import os, json, httpx, re
from typing import List, Optional, Dict, Any
from datetime import datetime
from google.colab import userdata

# LangChain
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

# Gemini via LangChain + SDK nativo
from langchain_google_genai import ChatGoogleGenerativeAI
import google.generativeai as genai  # SDK oficial Google

# ======== PARAMS (edite aqui) ========
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
DUMPLING_API_KEY = userdata.get('DUMPLING_API_KEY')
DUMPLING_KB_ID = userdata.get('DUMPLING_KB_ID')
GEMINI_MODEL = "gemini-2.5-flash"  # @param {type:"string"}
TEMPERATURE = 0.6  # @param {type:"number"}
MAX_TOKENS = 2048
MAX_REFINEMENT_ATTEMPTS = 2

assert GOOGLE_API_KEY, "Defina GOOGLE_API_KEY"
assert DUMPLING_API_KEY, "Defina DUMPLING_API_KEY"
assert DUMPLING_KB_ID, "Defina DUMPLING_KB_ID"

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

# ======== Dumpling config ========
DUMPLING_BASE = "https://app.dumplingai.com/api/v1"
DUMPLING_HEADERS = {
    "Authorization": f"Bearer {DUMPLING_API_KEY}",
    "Content-Type": "application/json"
}

print("✅ Chaves configuradas")

✅ Chaves configuradas


In [39]:
# Definição do Estado do Grafo (AgentState)
from typing import TypedDict, Optional, Dict, Any

class AgentState(TypedDict):
    """Define a estrutura de dados que será compartilhada e modificada pelos nós do grafo."""
    briefing: Dict
    contexto_rag: str
    dores_promessas: Optional[Dict]
    objecoes_quebras: Optional[Dict]
    headlines_angulos: Optional[Dict]
    contexto_enriquecido: Optional[str]
    copy_por_canal: Optional[Dict]
    revisao_critico: Optional[str]
    tentativas_refinamento: int

In [22]:
import re
import json

def force_json(llm_output: Any) -> Dict:
    """
    Extrai e analisa de forma robusta um bloco JSON da saída de um LLM.
    A saída pode ser um objeto de mensagem, string, etc.
    """
    # Se a saída já for um dicionário, retorne-a.
    if isinstance(llm_output, dict):
        return llm_output

    # Extrai o conteúdo da string, seja de um objeto de mensagem ou de uma string bruta.
    content_str = ""
    if hasattr(llm_output, 'content'):
        content_str = llm_output.content
    elif isinstance(llm_output, str):
        content_str = llm_output
    else:
        return {"error": "Tipo de entrada inválido", "content": str(llm_output)}

    # Procura por um bloco de código JSON na string.
    match = re.search(r"```json\s*([\s\S]*?)\s*```", content_str)

    json_str = ""
    if match:
        json_str = match.group(1)
    else:
        # Se não houver bloco de código, assume que a string inteira pode ser um JSON.
        # Isso adiciona flexibilidade caso o LLM retorne apenas o JSON.
        json_str = content_str

    try:
        # Tenta analisar a string JSON.
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        # Se a análise falhar, retorna um erro estruturado para facilitar a depuração.
        print(f"Erro ao decodificar JSON: {e}")
        return {"error": "Falha na decodificação do JSON", "raw_content": json_str}

print("✅ Função 'force_json' para extração robusta de JSON pronta.")

✅ Função 'force_json' para extração robusta de JSON pronta.


In [23]:

# 🌐 Cliente HTTP compartilhado com retry/backoff
import time, random
from typing import Callable

HTTP_TIMEOUT = 60.0
HTTP_MAX_RETRIES = 3
HTTP_BACKOFF_BASE = 0.8

def _with_retries(fn: Callable, *args, **kwargs):
    last_exc = None
    for attempt in range(1, HTTP_MAX_RETRIES+1):
        try:
            return fn(*args, **kwargs)
        except httpx.HTTPStatusError as e:
            status = e.response.status_code if e.response is not None else -1
            if status in (429, 500, 502, 503, 504):
                delay = (HTTP_BACKOFF_BASE ** attempt) + random.random()
                print(f"[retry {attempt}] status={status} aguardando {delay:.2f}s...")
                time.sleep(delay)
                last_exc = e
                continue
            raise
        except (httpx.TimeoutException, httpx.TransportError) as e:
            delay = (HTTP_BACKOFF_BASE ** attempt) + random.random()
            print(f"[retry {attempt}] timeout/transport aguardando {delay:.2f}s...")
            time.sleep(delay)
            last_exc = e
            continue
    if last_exc:
        raise last_exc

shared_client = httpx.Client(timeout=HTTP_TIMEOUT)
print("✅ HTTP client com retries pronto")


✅ HTTP client com retries pronto


In [24]:

# ✂️ Chunking para conteúdos longos ao indexar
def dumpling_add_text_chunked(knowledge_base_id: str, name_prefix: str, content: str, max_chars: int = 6000):
    parts = []
    for i in range(0, len(content), max_chars):
        parts.append(content[i:i+max_chars])
    added = []
    for i, p in enumerate(parts, 1):
        suffix = f"{i:02d}" if len(parts) > 1 else "01"
        added.append(dumpling_add_to_kb(knowledge_base_id, f"{name_prefix} (parte {suffix})", p))
    return added

print("✅ Função de chunking pronta")


✅ Função de chunking pronta


In [25]:
# 🧩 Funções auxiliares – DumplingAI (httpx)
class DumplingError(RuntimeError):
    pass

def dumpling_add_to_kb(knowledge_base_id: str, name: str, content: str) -> Dict[str, Any]:
    """
    POST /knowledge-bases/add
    Body: { knowledgeBaseId, name, content }
    Docs: https://docs.dumplingai.com/api-reference/endpoint/add-to-knowledge-base
    """
    url = f"{DUMPLING_BASE}/knowledge-bases/add"
    payload = {
        "knowledgeBaseId": knowledge_base_id,
        "name": name,
        "content": content,
    }
    r = httpx.post(url, headers=DUMPLING_HEADERS, json=payload, timeout=60)
    if r.status_code >= 400:
        raise DumplingError(f"Add KB falhou: {r.status_code} – {r.text}")
    return r.json()

def dumpling_search_kb(knowledge_base_id: str, query: str, result_count: int = 8) -> List[Dict[str, Any]]:
    """
    POST /knowledge-bases/query
    Body: { knowledgeBaseId, query, resultCount }
    Docs: https://docs.dumplingai.com/api-reference/endpoint/search-knowledge-base
    """
    url = f"{DUMPLING_BASE}/knowledge-bases/query"
    payload = {
        "knowledgeBaseId": knowledge_base_id,
        "query": query,
        "resultCount": max(1, min(int(result_count), 25))
    }
    r = httpx.post(url, headers=DUMPLING_HEADERS, json=payload, timeout=60)
    if r.status_code >= 400:
        raise DumplingError(f"Query KB falhou: {r.status_code} – {r.text}")
    return r.json()

print("✅ Dumpling helpers prontos")

✅ Dumpling helpers prontos


In [26]:
# 🔎 Retriever customizado
class DumplingRetriever(BaseRetriever):
    knowledge_base_id: str
    top_k: int = 8

    def _get_relevant_documents(
        self,
        query: str,
        *,
        run_manager: Optional[CallbackManagerForRetrieverRun] = None,
    ) -> List[Document]:
        hits = dumpling_search_kb(self.knowledge_base_id, query, self.top_k)
        docs = []
        for h in hits:
            docs.append(Document(
                page_content=h.get("content") or "",
                metadata={
                    "id": h.get("id"),
                    "resource_name": h.get("resource_name"),
                    "similarity": h.get("similarity"),
                }
            ))
        return docs

retriever = DumplingRetriever(knowledge_base_id=DUMPLING_KB_ID, top_k=8)
print("✅ Retriever pronto")

✅ Retriever pronto


In [27]:
BRIEFING_JSON = r'''
{
  "briefing_lancamento": {
    "infoproduto": {
      "nome": "Mentoria de Desenvolvimento Inteligente",
      "produtor": "Mauricio Issei",
      "preco": 99997.00,
      "formato": "Mentoria Individual",
      "descricao": "Mentoria individual para desenvolver arquiteturas de soluções complexas"
    },
    "publico_alvo": {
      "demografia": "Empreendedores digitais iniciantes, profissionais de tecnologia",
      "problema_principal": "Dificuldade em resolver problemas, falta de método para desenvolvimento, baixo faturamento.",
      "transformacao_principal": "Criar soluções de alta qualidade e escalável do zero, alcançando faturamento de 6 ou 7 dígitos.",
      "objecoes_comuns": [
        "Não tenho conhecimento suficiente",
        "O preço é muito alto",
        "Não tenho tempo para aplicar o método"
      ]
    },
    "posicionamento": {
      "diferencial_competitivo": "Único método que combina estratégia de arquitetura de soluções com o poder do desenvolvimento com ferramentas de IA.",
      "tom_de_voz": "Autoridade, inspirador, prático",
      "gatilhos_mentais": [
        "Autoridade",
        "Prova Social",
        "Escassez",
        "Reciprocidade"
      ]
    },
    "estrategia_lancamento": {
      "tipo_lancamento": "Semente",
      "meta_campanha": "Vender 500 unidades e faturar R$ 500.000",
      "datas_chave": {
        "inicio_campanha": "2025-09-15",
        "abertura_carrinho": "2025-09-22",
        "fechamento_carrinho": "2025-09-29"
      },
      "canais": [
        "Email Marketing",
        "Meta Ads",
        "Instagram Stories",
        "YouTube"
      ]
    }
  }
}
'''

briefing = json.loads(BRIEFING_JSON)
print("✅ Briefing carregado")

✅ Briefing carregado


In [28]:

# ✅ Validação do briefing com Pydantic
from pydantic import BaseModel, Field, ValidationError
from typing import List, Optional

class Infoproduto(BaseModel):
    nome: str
    produtor: str
    preco: float
    formato: str
    descricao: str

class PublicoAlvo(BaseModel):
    demografia: str
    problema_principal: str
    transformacao_principal: str
    objecoes_comuns: List[str]

class Posicionamento(BaseModel):
    diferencial_competitivo: str
    tom_de_voz: str
    gatilhos_mentais: List[str]

class Datas(BaseModel):
    inicio_campanha: str
    abertura_carrinho: str
    fechamento_carrinho: str

class Estrategia(BaseModel):
    tipo_lancamento: str
    meta_campanha: str
    datas_chave: Datas
    canais: List[str]

class BriefingLancamento(BaseModel):
    infoproduto: Infoproduto
    publico_alvo: PublicoAlvo
    posicionamento: Posicionamento
    estrategia_lancamento: Estrategia

class BriefingEnvelope(BaseModel):
    briefing_lancamento: BriefingLancamento

try:
    _validated = BriefingEnvelope(**briefing)
    print("✅ Briefing validado com Pydantic")
except ValidationError as e:
    print("❌ Erro de validação no briefing:\n", e)
    raise


✅ Briefing validado com Pydantic


In [29]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import timezone

# Estilo para os campos de texto, para garantir que as descrições não sejam cortadas
style = {'description_width': 'initial'}
layout = widgets.Layout(width='95%')

#--------------------------------------------------------------------
# 1. WIDGETS PARA "INFOPRODUTO"
#--------------------------------------------------------------------
info_data = briefing['briefing_lancamento']['infoproduto']
w_info_nome = widgets.Text(value=info_data['nome'], description='Nome:', style=style, layout=layout)
w_info_produtor = widgets.Text(value=info_data['produtor'], description='Produtor:', style=style, layout=layout)
w_info_preco = widgets.FloatText(value=info_data['preco'], description='Preço (R$):', style=style, layout=layout)
w_info_formato = widgets.Text(value=info_data['formato'], description='Formato:', style=style, layout=layout)
w_info_descricao = widgets.Textarea(value=info_data['descricao'], description='Descrição:', style=style, layout=layout)

#--------------------------------------------------------------------
# 2. WIDGETS PARA "PÚBLICO ALVO"
#--------------------------------------------------------------------
pa_data = briefing['briefing_lancamento']['publico_alvo']
w_pa_demografia = widgets.Textarea(value=pa_data['demografia'], description='Demografia:', style=style, layout=layout)
w_pa_problema = widgets.Textarea(value=pa_data['problema_principal'], description='Problema Principal:', style=style, layout=layout)
w_pa_transformacao = widgets.Textarea(value=pa_data['transformacao_principal'], description='Transformação Principal:', style=style, layout=layout)
w_pa_objecoes = widgets.Textarea(value='\n'.join(pa_data['objecoes_comuns']), description='Objeções Comuns (uma por linha):', style=style, layout=layout)

#--------------------------------------------------------------------
# 3. WIDGETS PARA "POSICIONAMENTO"
#--------------------------------------------------------------------
pos_data = briefing['briefing_lancamento']['posicionamento']
w_pos_diferencial = widgets.Textarea(value=pos_data['diferencial_competitivo'], description='Diferencial Competitivo:', style=style, layout=layout)
w_pos_tom = widgets.Text(value=pos_data['tom_de_voz'], description='Tom de Voz:', style=style, layout=layout)
w_pos_gatilhos = widgets.Textarea(value='\n'.join(pos_data['gatilhos_mentais']), description='Gatilhos Mentais (um por linha):', style=style, layout=layout)

#--------------------------------------------------------------------
# 4. WIDGETS PARA "ESTRATÉGIA DE LANÇAMENTO"
#--------------------------------------------------------------------
estr_data = briefing['briefing_lancamento']['estrategia_lancamento']
w_estr_tipo = widgets.Text(value=estr_data['tipo_lancamento'], description='Tipo de Lançamento:', style=style, layout=layout)
w_estr_meta = widgets.Text(value=estr_data['meta_campanha'], description='Meta da Campanha:', style=style, layout=layout)
w_estr_inicio = widgets.DatePicker(description='Início da Campanha:', value=datetime.strptime(estr_data['datas_chave']['inicio_campanha'], '%Y-%m-%d'), style=style)
w_estr_abertura = widgets.DatePicker(description='Abertura do Carrinho:', value=datetime.strptime(estr_data['datas_chave']['abertura_carrinho'], '%Y-%m-%d'), style=style)
w_estr_fechamento = widgets.DatePicker(description='Fechamento do Carrinho:', value=datetime.strptime(estr_data['datas_chave']['fechamento_carrinho'], '%Y-%m-%d'), style=style)
w_estr_canais = widgets.Textarea(value='\n'.join(estr_data['canais']), description='Canais (um por linha):', style=style, layout=layout)

#--------------------------------------------------------------------
# MONTAGEM DA INTERFACE COM ACORDEÃO
#--------------------------------------------------------------------
# Agrupa os widgets de cada seção em caixas verticais (VBox)
box_infoproduto = widgets.VBox([w_info_nome, w_info_produtor, w_info_preco, w_info_formato, w_info_descricao])
box_publico_alvo = widgets.VBox([w_pa_demografia, w_pa_problema, w_pa_transformacao, w_pa_objecoes])
box_posicionamento = widgets.VBox([w_pos_diferencial, w_pos_tom, w_pos_gatilhos])
box_estrategia = widgets.VBox([w_estr_tipo, w_estr_meta, w_estr_inicio, w_estr_abertura, w_estr_fechamento, w_estr_canais])

# Cria o acordeão e define as seções
accordion = widgets.Accordion(children=[box_infoproduto, box_publico_alvo, box_posicionamento, box_estrategia])
accordion.set_title(0, '🚀 Infoproduto')
accordion.set_title(1, '🎯 Público Alvo')
accordion.set_title(2, '📣 Posicionamento')
accordion.set_title(3, '🗓️ Estratégia de Lançamento')

# Botão para salvar as alterações
button = widgets.Button(description="✅ Atualizar Briefing", button_style='success', layout=widgets.Layout(width='200px', margin='10px 0 0 0'))
output = widgets.Output() # Área para exibir mensagens de status

# Função que será chamada quando o botão for clicado
def on_button_clicked(b):
    # Atualiza o dicionário 'briefing' com os valores dos widgets
    b_lanc = briefing['briefing_lancamento']
    b_lanc['infoproduto']['nome'] = w_info_nome.value
    b_lanc['infoproduto']['produtor'] = w_info_produtor.value
    b_lanc['infoproduto']['preco'] = w_info_preco.value
    b_lanc['infoproduto']['formato'] = w_info_formato.value
    b_lanc['infoproduto']['descricao'] = w_info_descricao.value

    b_lanc['publico_alvo']['demografia'] = w_pa_demografia.value
    b_lanc['publico_alvo']['problema_principal'] = w_pa_problema.value
    b_lanc['publico_alvo']['transformacao_principal'] = w_pa_transformacao.value
    b_lanc['publico_alvo']['objecoes_comuns'] = w_pa_objecoes.value.split('\n')

    b_lanc['posicionamento']['diferencial_competitivo'] = w_pos_diferencial.value
    b_lanc['posicionamento']['tom_de_voz'] = w_pos_tom.value
    b_lanc['posicionamento']['gatilhos_mentais'] = w_pos_gatilhos.value.split('\n')

    b_lanc['estrategia_lancamento']['tipo_lancamento'] = w_estr_tipo.value
    b_lanc['estrategia_lancamento']['meta_campanha'] = w_estr_meta.value
    b_lanc['estrategia_lancamento']['datas_chave']['inicio_campanha'] = w_estr_inicio.value.strftime('%Y-%m-%d')
    b_lanc['estrategia_lancamento']['datas_chave']['abertura_carrinho'] = w_estr_abertura.value.strftime('%Y-%m-%d')
    b_lanc['estrategia_lancamento']['datas_chave']['fechamento_carrinho'] = w_estr_fechamento.value.strftime('%Y-%m-%d')
    b_lanc['estrategia_lancamento']['canais'] = w_estr_canais.value.split('\n')

    # Exibe uma mensagem de sucesso
    with output:
        clear_output()
        print(f"✔️ Briefing atualizado com sucesso às {datetime.now().strftime('%H:%M:%S')}!")
        # Opcional: Imprime o JSON atualizado para verificação
        # print(json.dumps(briefing, indent=2, ensure_ascii=False))


# Associa a função ao evento de clique do botão
button.on_click(on_button_clicked)

# Exibe a interface
display(accordion, button, output)

Accordion(children=(VBox(children=(Text(value='Mentoria de Desenvolvimento Inteligente', description='Nome:', …

Button(button_style='success', description='✅ Atualizar Briefing', layout=Layout(margin='10px 0 0 0', width='2…

Output()

In [30]:
def canonicalize_briefing_to_text(briefing: Dict[str, Any]) -> str:
    b    = briefing.get("briefing_lancamento", {})
    inf  = b.get("infoproduto", {})
    pub  = b.get("publico_alvo", {})
    pos  = b.get("posicionamento", {})
    est  = b.get("estrategia_lancamento", {})
    datas= est.get("datas_chave", {})

    linhas = []
    linhas.append("# Briefing de Lançamento — Canonicalizado")
    linhas.append("## Produto")
    linhas.append(f"Nome: {inf.get('nome','')} | Produtor: {inf.get('produtor','')} | Preço: {inf.get('preco','')} | Formato: {inf.get('formato','')}")
    linhas.append(f"Descrição: {inf.get('descricao','')}")
    linhas.append("\n## Público-alvo & Persona")
    linhas.append(f"Demografia/Psicografia: {pub.get('demografia','')}")
    linhas.append(f"Dor principal: {pub.get('problema_principal','')}")
    linhas.append(f"Transformação: {pub.get('transformacao_principal','')}")
    if pub.get("objecoes_comuns"):
        linhas.append("Objeções comuns:")
        for o in pub["objecoes_comuns"]:
            linhas.append(f"- {o}")
    linhas.append("\n## Posicionamento & Diferencial")
    linhas.append(f"USP: {pos.get('diferencial_competitivo','')}")
    linhas.append(f"Tom de voz: {pos.get('tom_de_voz','')}")
    if pos.get("gatilhos_mentais"):
        linhas.append("Gatilhos prioritários: " + ", ".join(pos["gatilhos_mentais"]))
    linhas.append("\n## Estratégia de Lançamento")
    linhas.append(f"Tipo: {est.get('tipo_lancamento','')} | Meta: {est.get('meta_campanha','')}")
    linhas.append(f"Período/Datas: início={datas.get('inicio_campanha','')} | abertura={datas.get('abertura_carrinho','')} | fechamento={datas.get('fechamento_carrinho','')}")
    if est.get("canais"):
        linhas.append("Canais: " + ", ".join(est["canais"]))
    return "\n".join(linhas)

canonical_text = canonicalize_briefing_to_text(briefing)
kb_name = f"Briefing — {datetime.now(timezone.utc).isoformat()}"

added = dumpling_add_text_chunked(DUMPLING_KB_ID, kb_name, canonical_text)
print(f"✅ Briefing indexado em {len(added)} parte(s)")

✅ Briefing indexado em 1 parte(s)


In [40]:
llm = ChatGoogleGenerativeAI(
    model=GEMINI_MODEL,
    temperature=TEMPERATURE
)

SYSTEM_BASE = (
    "Você é parte de uma REDE DE AGENTES especialista em copy para lançamentos de infoprodutos. "
    "Use o TOM DE VOZ do briefing. Foque em clareza, estratégia e ativos prontos para usar. "
    "Respeite a persona (dores, desejos) e o posicionamento (USP)."
)

def join_docs(docs: List[Document]) -> str:
    blocos = []
    for i, d in enumerate(docs, 1):
        src = d.metadata.get("resource_name") or d.metadata.get("id") or f"doc{i}"
        blocos.append(f"[Fonte: {src}]\n{d.page_content}")
    return "\n\n".join(blocos)

def build_rag_context(brief: Dict[str, Any]) -> str:
    b   = brief.get("briefing_lancamento", {})
    pub = b.get("publico_alvo", {})
    pos = b.get("posicionamento", {})
    est = b.get("estrategia_lancamento", {})

    queries = []
    if pub.get("problema_principal"):
        queries.append(pub["problema_principal"])
    if pub.get("transformacao_principal"):
        queries.append(pub["transformacao_principal"])
    if pos.get("diferencial_competitivo"):
        queries.append(pos["diferencial_competitivo"])
    if est.get("tipo_lancamento"):
        queries.append(f"táticas de lançamento {est['tipo_lancamento']}")

    if not queries:
        queries = ["briefing do produto"]

    docs_all: List[Document] = []
    for q in queries:
        docs_all.extend(retriever.get_relevant_documents(q))

    # dedup simples
    seen, uniq = set(), []
    for d in docs_all:
        key = (d.page_content[:120], d.metadata.get("id"))
        if key not in seen:
            seen.add(key)
            uniq.append(d)

    return join_docs(uniq[:12])

contexto = build_rag_context(briefing)
briefing_str = json.dumps(briefing, ensure_ascii=False, indent=2)

# ===== Agente 1: Dores & Promessas =====
prompt_dores = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Gere DORES priorizadas e PROMESSAS (transformações)."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: dores[], promessas[].")
])
chain_dores = prompt_dores | llm

# ===== Agente 2: Objeções & Quebras =====
prompt_obj = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Liste as OBJEÇÕES mais prováveis e QUEBRAS (respostas estratégicas)."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: objecoes[], quebras[].")
])
chain_obj = prompt_obj | llm

# ===== Agente 3: Headlines & Ângulos =====
prompt_head = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Proponha HEADLINES e ÂNGULOS criativos prontos para testes."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: headlines[], angulos[].")
])
chain_head = prompt_head | llm

# ===== Agente 4: Adaptação por Canais =====
prompt_canais = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Adapte a mensagem para Canais: Email, Stories IG, Meta Ads, YouTube/VSL."),
    ("human", "Briefing (JSON):\n{briefing}\n\nContexto RAG:\n{contexto}\n\nRetorne JSON com campos: email.sequence[], stories.scripts[], ads.variacoes[], vsl.outline[].")
])
chain_canais = prompt_canais | llm

print("✅ Prompts e cadeias prontos")

✅ Prompts e cadeias prontos


In [32]:
# 3. Conversão dos Agentes em Nós
def node_dores_promessas(state: AgentState) -> Dict[str, Any]:
    """Executa o agente de dores e promessas."""
    print("Executando nó de Dores & Promessas...")
    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    result = chain_dores.invoke({
        "briefing": briefing_str,
        "contexto": state['contexto_rag']
    })
    return {"dores_promessas": force_json(result)}

def node_objecoes_quebras(state: AgentState) -> Dict[str, Any]:
    """Executa o agente de objeções e quebras."""
    print("Executando nó de Objeções & Quebras...")
    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    result = chain_obj.invoke({
        "briefing": briefing_str,
        "contexto": state['contexto_rag']
    })
    return {"objecoes_quebras": force_json(result)}

def node_headlines_angulos(state: AgentState) -> Dict[str, Any]:
    """Executa o agente de headlines e ângulos."""
    print("Executando nó de Headlines & Ângulos...")
    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    result = chain_head.invoke({
        "briefing": briefing_str,
        "contexto": state['contexto_rag']
    })
    return {"headlines_angulos": force_json(result)}

In [33]:
# Define o estado inicial para a execução
initial_state = AgentState(
    briefing=briefing,
    contexto_rag=contexto,
    dores_promessas=None,
    objecoes_quebras=None,
    headlines_angulos=None,
    contexto_enriquecido=None,
    copy_por_canal=None,
    revisao_critico=None,
    tentativas_refinamento=0
)

# Invoca o grafo
print("\n🚀 Iniciando a execução do grafo de agentes (com etapa de HTML)...\n")
final_state = app.invoke(initial_state)

print("\n\n✅ Execução do grafo concluída!\n")
print("="*50)

# Salva os resultados finais (JSON e HTML)
os.makedirs("outputs_langgraph", exist_ok=True)

# Salva os assets de copy em JSON
assets_json_path = "outputs_langgraph/assets_final.json"
with open(assets_json_path, "w", encoding="utf-8") as f:
    json.dump(final_state['copy_por_canal'], f, ensure_ascii=False, indent=2)
print(f"📄 Assets de copy salvos em '{assets_json_path}'")



🚀 Iniciando a execução do grafo de agentes (com etapa de HTML)...

Executando nó de Dores & Promessas...
Executando nó de Headlines & Ângulos...
Executando nó de Objeções & Quebras...
Erro ao decodificar JSON: Expecting value: line 1 column 1 (char 0)
Executando nó Consolidador...
Executando nó de Adaptação por Canais...
Executando nó Crítico Revisor...
Executando nó de Decisão Pós-Crítica...
Decisão: Copy APROVADA. Prosseguindo para a geração do HTML. (Tentativas: 1)
Executando nó do Especialista em Apresentação HTML...
Executando nó do Desenvolvedor Web Tailwind...


✅ Execução do grafo concluída!

📄 Assets de copy salvos em 'outputs_langgraph/assets_final.json'


In [34]:
from langgraph.graph import StateGraph, END
from functools import partial
from typing import Callable

# -----------------------------------
# 4. Nós Colaborativos (Consolidador, Adaptação, Crítico)
# -----------------------------------

def node_consolidador(state: AgentState) -> Dict[str, Any]:
    """Concatena as saídas dos nós iniciais para criar um contexto enriquecido."""
    print("Executando nó Consolidador...")
    contexto_enriquecido = (
        "## Dores e Promessas\n"
        f"{json.dumps(state['dores_promessas'], ensure_ascii=False, indent=2)}\n\n"
        "## Objeções e Quebras\n"
        f"{json.dumps(state['objecoes_quebras'], ensure_ascii=False, indent=2)}\n\n"
        "## Headlines e Ângulos\n"
        f"{json.dumps(state['headlines_angulos'], ensure_ascii=False, indent=2)}"
    )
    return {"contexto_enriquecido": contexto_enriquecido}

# Prompt e cadeia para o agente de adaptação (sem alterações)
prompt_canais_refinado = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_BASE + " Adapte a mensagem para Canais usando o CONTEXTO ENRIQUECIDO. Se houver uma REVISÃO, aplique as melhorias sugeridas."),
    ("human",
     "Briefing Original (JSON):\n{briefing}\n\n"
     "Contexto Enriquecido (gerado pelos agentes anteriores):\n{contexto_enriquecido}\n\n"
     "Revisão Anterior (se houver):\n{revisao_critico}\n\n"
     "Gere a copy adaptada para os canais. Retorne JSON com campos: email.sequence[], stories.scripts[], ads.variacoes[], vsl.outline[].")
])
chain_canais_refinado = prompt_canais_refinado | llm

def node_adaptacao_canais(state: AgentState) -> Dict[str, Any]:
    """Executa o agente de adaptação por canais usando o contexto enriquecido."""
    print("Executando nó de Adaptação por Canais...")
    tentativas = state.get('tentativas_refinamento', 0) + 1
    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    revisao = state.get('revisao_critico') or "Nenhuma"
    result = chain_canais_refinado.invoke({
        "briefing": briefing_str,
        "contexto_enriquecido": state['contexto_enriquecido'],
        "revisao_critico": revisao
    })
    return {"copy_por_canal": force_json(result), "tentativas_refinamento": tentativas}

# Prompt e cadeia para o agente crítico (sem alterações)
prompt_critico = ChatPromptTemplate.from_messages([
    ("system",
     "Você é um CRÍTICO DE MARKETING sênior e exigente. Sua tarefa é revisar a copy gerada para um lançamento. "
     "Seu feedback deve ser direto, acionável e baseado estritamente no briefing original. "
     "Se a copy estiver excelente e alinhada, responda apenas com a palavra 'APROVADO'. "
     "Se precisar de ajustes, comece com a palavra 'REFINAR:' e liste em bullets as mudanças necessárias. Seja específico. Ex: 'REFINAR: O tom de voz no email está muito informal, mude para algo mais de autoridade. Os stories precisam de um CTA mais claro.'"),
    ("human",
     "Briefing Original (JSON):\n{briefing}\n\n"
     "Copy Gerada para Revisão (JSON):\n{copy_por_canal}\n\n"
     "Avalie a copy. Responda 'APROVADO' ou 'REFINAR:' com suas sugestões.")
])
chain_critico = prompt_critico | llm


def node_critico_revisor(state: AgentState) -> Dict[str, Any]:
    """Executa o agente crítico para revisar a copy gerada."""
    print("Executando nó Crítico Revisor...")
    briefing_str = json.dumps(state['briefing'], ensure_ascii=False, indent=2)
    copy_str = json.dumps(state['copy_por_canal'], ensure_ascii=False, indent=2)
    result = chain_critico.invoke({
        "briefing": briefing_str,
        "copy_por_canal": copy_str
    })
    return {"revisao_critico": result.content}


# -----------------------------------
# 6. Construção e Conexão do Grafo Atualizado
# -----------------------------------

def decidir_pos_critica(state: AgentState) -> str:
    """Decide o próximo passo após a revisão do crítico."""
    print("Executando nó de Decisão Pós-Crítica...")
    revisao = state.get('revisao_critico', '')
    tentativas = state.get('tentativas_refinamento', 0)

    if "APROVADO" in revisao or tentativas >= MAX_REFINEMENT_ATTEMPTS:
        print(f"Decisão: Copy APROVADA.")
        return "APROVADA"
    else:
        print(f"Decisão: Refinar (Tentativa {tentativas + 1}).")
        return "refinar"

# Montagem do grafo
workflow = StateGraph(AgentState)

# Adiciona todos os nós, incluindo os novos
workflow.add_node("dores_promessas", node_dores_promessas)
workflow.add_node("objecoes_quebras", node_objecoes_quebras)
workflow.add_node("headlines_angulos", node_headlines_angulos)
workflow.add_node("consolidador", node_consolidador)
workflow.add_node("adaptacao_canais", node_adaptacao_canais)
workflow.add_node("critico_revisor", node_critico_revisor)
workflow.add_node("gerador_estrutura_html", node_gerador_estrutura_html)

# Define os pontos de partida (execução paralela)
workflow.set_entry_point("dores_promessas")
workflow.set_entry_point("objecoes_quebras")
workflow.set_entry_point("headlines_angulos")

# Define as arestas do fluxo principal
workflow.add_edge("dores_promessas", "consolidador")
workflow.add_edge("objecoes_quebras", "consolidador")
workflow.add_edge("headlines_angulos", "consolidador")
workflow.add_edge("consolidador", "adaptacao_canais")
workflow.add_edge("adaptacao_canais", "critico_revisor")

# Adiciona a aresta condicional
workflow.add_conditional_edges(
    "critico_revisor",
    decidir_pos_critica,
    {
        "refinar": "adaptacao_canais",
        "APROVADA": END
    }
)

print("✅ Grafo atualizado com os agentes está construído.")

# Compila o workflow
app = workflow.compile()
print("✅ Workflow compilado e pronto para execução.")

✅ Grafo atualizado com os agentes de geração de HTML está construído.
✅ Workflow compilado e pronto para execução.


In [38]:
# 💾 Normalizar/Salvar assets com nomes determinísticos (produto + timestamp)
from pydantic import BaseModel, Field
import re, os
from datetime import timezone
import json # Ensure json is imported

class DoresPromessas(BaseModel):
    dores: list = Field(default_factory=list)
    promessas: list = Field(default_factory=list)

class ObjQuebras(BaseModel):
    objecoes: list = Field(default_factory=list)
    quebras: list = Field(default_factory=list)

class HeadlinesAngulos(BaseModel):
    headlines: list = Field(default_factory=list)
    angulos: list = Field(default_factory=list)

class PorCanais(BaseModel):
    email: dict = Field(default_factory=dict)
    stories: dict = Field(default_factory=dict)
    ads: dict = Field(default_factory=dict)
    vsl: dict = Field(default_factory=dict)

# Helper function to extract and parse content from 'raw' key
def extract_and_parse_raw(data):
    if isinstance(data, dict) and 'raw' in data:
        try:
            print(f"Attempting to parse JSON from raw: {data['raw'][:100]}...") # Debug print
            parsed_data = json.loads(data['raw'])
            print("Parsing successful.") # Debug print
            return parsed_data
        except (json.JSONDecodeError, TypeError) as e:
            print(f"JSON parsing failed: {e}") # Debug print
            return {"parsing_error": data['raw']} # Indicate parsing failure
    print("No 'raw' key found or data is not a dict.") # Debug print
    return data # Return data as is if no 'raw' key or not a dict

# Coerção para esquemas (tolerante a chaves com acentos/nome alternativo)
def coerce(model_cls, data):
    print(f"Attempting to coerce data for {model_cls.__name__}: {data}") # Debug print
    try:
        coerced_data = model_cls(**data).model_dump()
        print(f"Coercion successful for {model_cls.__name__}") # Debug print
        return coerced_data
    except Exception as e:
        print(f"Coercion failed for {model_cls.__name__}: {e} with data {data}") # Debug print
        # tenta mapear variantes comuns
        if model_cls is DoresPromessas:
            parsed_data = {
                "dores": data.get("dores") or data.get("pains") or [],
                "promessas": data.get("promessas") or data.get("promises") or [],
            }
            try:
                 return model_cls(**parsed_data).model_dump()
            except Exception:
                 return {"coercion_error": f"Could not map fields for {model_cls.__name__}"}
        if model_cls is ObjQuebras:
            parsed_data = {
                "objecoes": data.get("objecoes") or data.get("objeções") or data.get("objections") or [],
                "quebras": data.get("quebras") or data.get("rebuttals") or [],
            }
            try:
                 return model_cls(**parsed_data).model_dump()
            except Exception:
                 return {"coercion_error": f"Could not map fields for {model_cls.__name__}"}
        if model_cls is HeadlinesAngulos:
            parsed_data = {
                "headlines": data.get("headlines") or [],
                "angulos": data.get("angulos") or data.get("angles") or [],
            }
            try:
                 return model_cls(**parsed_data).model_dump()
            except Exception:
                 return {"coercion_error": f"Could not map fields for {model_cls.__name__}"}
        if model_cls is PorCanais:
            parsed_data = {
                "email":   data.get("email")   or {},
                "stories": data.get("stories") or {},
                "ads":     data.get("ads")     or {},
                "vsl":     data.get("vsl")     or {},
            }
            try:
                 return model_cls(**parsed_data).model_dump()
            except Exception:
                 return {"coercion_error": f"Could not map fields for {model_cls.__name__}"}
        return {"coercion_error": f"Unknown model type: {model_cls.__name__}"}


# Use the extract_and_parse_raw helper directly when populating raw
print("\n--- Populating raw dictionary ---") # Debug print
raw = {
    "dores_promessas":  extract_and_parse_raw(final_state.get('dores_promessas', {})),
    "objeções_quebras": extract_and_parse_raw(final_state.get('objecoes_quebras', {})),
    "headlines_angulos":extract_and_parse_raw(final_state.get('headlines_angulos', {})),
    "por_canais":       extract_and_parse_raw(final_state.get('copy_por_canal', {})),
}
print("\n--- Raw dictionary populated ---") # Debug print
print("Raw dictionary content:", json.dumps(raw, indent=2, ensure_ascii=False)) # Debug print

# Now coerce should work on potentially parsed data
print("\n--- Coercing raw data into assets ---") # Debug print
assets = {
    "dores_promessas":  coerce(DoresPromessas,  raw["dores_promessas"]),
    "objeções_quebras": coerce(ObjQuebras,      raw["objeções_quebras"]),
    "headlines_angulos":coerce(HeadlinesAngulos,raw["headlines_angulos"]),
    "por_canais":       coerce(PorCanais,       raw["por_canais"]),
}
print("\n--- Assets dictionary populated ---") # Debug print
print("Assets dictionary content:", json.dumps(assets, indent=2, ensure_ascii=False)) # Debug print


produto = briefing["briefing_lancamento"]["infoproduto"]["nome"]
slug = re.sub(r"[^a-z0-9]+", "-", produto.lower()).strip("-") or "infoproduto"

stamp = datetime.now(timezone.utc).strftime("%Y%m%d-%H%M%S")
outdir = f"outputs/{slug}-{stamp}"
os.makedirs(outdir, exist_ok=True)

with open(f"{outdir}/assets.json", "w", encoding="utf-8") as f:
    json.dump(assets, f, ensure_ascii=False, indent=2)

with open(f"{outdir}/assets.md", "w", encoding="utf-8") as f:
    f.write("# Assets de Lançamento (gerados)\n\n")
    for k,v in assets.items():
        f.write(f"## {k}\n```json\n{json.dumps(v, ensure_ascii=False, indent=2)}\n```\n\n")

print(f"✅ Salvo em {outdir}/assets.json e {outdir}/assets.md")


--- Populating raw dictionary ---
No 'raw' key found or data is not a dict.
No 'raw' key found or data is not a dict.
No 'raw' key found or data is not a dict.
No 'raw' key found or data is not a dict.
No 'raw' key found or data is not a dict.

--- Raw dictionary populated ---
Raw dictionary content: {
  "dores_promessas": {
    "dores": [
      "Seu faturamento está estagnado, longe dos 6 ou 7 dígitos que você sabe que é capaz de alcançar, apesar de todo o seu potencial?",
      "Você se sente incapaz de resolver problemas complexos e arquitetar soluções que realmente escalem, perdendo tempo e energia em 'remendos'?",
      "A falta de um método claro e inteligente impede que suas ideias brilhantes se transformem em produtos digitais de alto impacto e lucratividade?",
      "A dificuldade em desenvolver soluções de alta qualidade é o gargalo que impede seu crescimento, mantendo você distante do faturamento que realmente merece?"
    ],
    "promessas": [
      "Domine a arte de arqui

In [36]:

# ⬆️ (Opcional) Enviar assets ao Knowledge Base
PUSH_ASSETS_TO_KB = True  # @param {type:"boolean"}
if PUSH_ASSETS_TO_KB:
    content_assets = open("outputs_langgraph/assets_final.json", "r", encoding="utf-8").read()
    resp_add_assets = dumpling_add_text_chunked(DUMPLING_KB_ID, "Assets Gerados", content_assets)
    print(f"✅ {len(resp_add_assets)} parte(s) enviada(s) à KB")
else:
    print("Pulando envio à KB (PUSH_ASSETS_TO_KB=False).")


✅ 4 parte(s) enviada(s) à KB


In [42]:
import json
import re

# ======================================================================================
# CÓDIGO DE CORREÇÃO PARA GERAR A APRESENTAÇÃO
# Substitua a célula de geração de Markdown existente por este código.
# ======================================================================================

def extract_content_from_final_state(data):
    """
    Robustly extracts the relevant content from the nested structure within final_state.
    It handles cases where the content might be a dictionary with a 'raw' key
    containing a JSON string, or a direct dictionary/string.
    """
    if isinstance(data, dict):
        if 'raw' in data and isinstance(data['raw'], str):
            try:
                # Attempt to parse the JSON string within 'raw'
                return json.loads(data['raw'])
            except (json.JSONDecodeError, TypeError):
                # If parsing fails, return the raw string
                return data['raw']
        # If it's a dictionary but no 'raw' key, return the dictionary itself
        return data
    # If it's not a dictionary (e.g., a string), return it directly
    return data


# --- Início da Geração da Apresentação em Markdown ---

presentation_md = "# Apresentação dos Assets de Lançamento\n\n"

# 1. Dores e Promessas
# Extract the content using the new helper function
dores_promessas_content = extract_content_from_final_state(final_state.get("dores_promessas", {}))
presentation_md += "## Dores e Promessas\n\n"
if isinstance(dores_promessas_content, dict):
    if dores_promessas_content.get("dores"):
        presentation_md += "**Dores:**\n"
        for dor in dores_promessas_content["dores"]:
            presentation_md += f"- {dor}\n"
    if dores_promessas_content.get("promessas"):
        presentation_md += "\n**Promessas (Transformações):**\n"
        for promessa in dores_promessas_content["promessas"]:
            presentation_md += f"- {promessa}\n"
elif isinstance(dores_promessas_content, str):
     presentation_md += f"```\n{dores_promessas_content}\n```\n"
presentation_md += "\n---\n\n"

# 2. Objeções e Quebras
# Extract the content using the new helper function
objecoes_quebras_content = extract_content_from_final_state(final_state.get("objecoes_quebras", {}))
presentation_md += "## Objeções e Quebras\n\n"
if isinstance(objecoes_quebras_content, dict):
    if objecoes_quebras_content.get("objecoes"):
        presentation_md += "**Objeções Comuns:**\n"
        for objecao in objecoes_quebras_content["objecoes"]:
            presentation_md += f"- {objecao}\n"
    if objecoes_quebras_content.get("quebras"):
        presentation_md += "\n**Quebras (Respostas Estratégicas):**\n"
        # Assuming 'quebras' is a list of dictionaries based on typical LLM output
        for quebra in objecoes_quebras_content["quebras"]:
            if isinstance(quebra, dict):
                presentation_md += f"- **Objeção:** {quebra.get('objecao', 'N/A')}\n"
                presentation_md += f"  - **Resposta:** {quebra.get('resposta_estrategica', 'N/A')}\n"
            else:
                 presentation_md += f"- {quebra}\n" # Fallback for unexpected format
elif isinstance(objecoes_quebras_content, str):
     presentation_md += f"```\n{objecoes_quebras_content}\n```\n"
presentation_md += "\n---\n\n"

# 3. Headlines e Ângulos
# Extract the content using the new helper function
headlines_angulos_content = extract_content_from_final_state(final_state.get("headlines_angulos", {}))
presentation_md += "## Headlines e Ângulos\n\n"
if isinstance(headlines_angulos_content, dict):
    if headlines_angulos_content.get("headlines"):
        presentation_md += "**Headlines:**\n"
        for headline in headlines_angulos_content["headlines"]:
            presentation_md += f"- {headline}\n"
    if headlines_angulos_content.get("angulos"):
        presentation_md += "\n**Ângulos:**\n"
        # Assuming 'angulos' is a list of dictionaries
        for angulo in headlines_angulos_content["angulos"]:
            if isinstance(angulo, dict):
                 presentation_md += f"- **Nome:** {angulo.get('nome', 'N/A')}\n"
                 if angulo.get('foco'): presentation_md += f"  - **Foco:** {angulo['foco']}\n"
                 if angulo.get('conteudo'): presentation_md += f"  - **Conteúdo:** {angulo['conteudo']}\n"
            else:
                 presentation_md += f"- {angulo}\n" # Fallback for unexpected format
elif isinstance(headlines_angulos_content, str):
     presentation_md += f"```\n{headlines_angulos_content}\n```\n"
presentation_md += "\n---\n\n"

# 4. Copy por Canal
# Extract the content using the new helper function
presentation_md += "## Copy por Canal\n\n"
final_copy_content = extract_content_from_final_state(final_state.get('copy_por_canal', {}))

canais_map = {
    "email": "Email Marketing",
    "stories": "Instagram Stories",
    "ads": "Meta Ads",
    "vsl": "YouTube/VSL Outline"
}

if isinstance(final_copy_content, dict):
    for canal_key, canal_name in canais_map.items():
        if final_copy_content.get(canal_key):
            presentation_md += f"### {canal_name}\n\n"
            content = final_copy_content[canal_key]
            if isinstance(content, list):
                 for i, item in enumerate(content, 1):
                     # Format list items nicely, assuming they are dictionaries
                     if isinstance(item, dict):
                         item_title = item.get('nome', f'{canal_name} Item {i}')
                         presentation_md += f"- **{item_title}**:\n"
                         for sub_key, sub_value in item.items():
                             if sub_key != 'nome':
                                  # Handle nested structures like 'roteiro' in stories
                                  if isinstance(sub_value, list):
                                      presentation_md += f"  - **{sub_key.replace('_', ' ').title()}:**\n"
                                      for sub_item in sub_value:
                                          if isinstance(sub_item, dict):
                                               presentation_md += f"    - **{sub_item.get('cena', 'Cena')}**: {sub_item.get('visual', '')}\n"
                                               if sub_item.get('audio'): presentation_md += f"      - Áudio: {sub_item['audio']}\n"
                                               if sub_item.get('interacao'): presentation_md += f"      - Interação: {sub_item['interacao']}\n"
                                          else:
                                               presentation_md += f"    - {sub_item}\n"
                                  elif isinstance(sub_value, dict):
                                      presentation_md += f"  - **{sub_key.replace('_', ' ').title()}:**\n"
                                      presentation_md += f"    ```json\n{json.dumps(sub_value, ensure_ascii=False, indent=2)}\n    ```\n"
                                  else:
                                       presentation_md += f"  - {sub_key.replace('_', ' ').title()}: {sub_value}\n"
                         presentation_md += "\n" # Add a newline after each item in the list
                     else:
                         presentation_md += f"- {item}\n\n" # Fallback for unexpected list item format
            elif isinstance(content, dict):
                 presentation_md += f"```json\n{json.dumps(content, ensure_ascii=False, indent=2)}\n```\n\n"
            else:
                 presentation_md += f"{content}\n\n"
elif isinstance(final_copy_content, str):
     presentation_md += f"```\n{final_copy_content}\n```\n\n"


# --- Finalização e Salvamento do Arquivo ---
presentation_md += "\n---\n\n"
presentation_md += "_Gerado automaticamente com LangGraph_"

# Define the output directory (assuming outdir is defined elsewhere, e.g., in jcqnR4RdJEwT)
# Fallback if outdir is not defined
if 'outdir' not in locals():
    print("Warning: 'outdir' variable not found. Saving presentation to a default location.")
    produto_name = briefing.get("briefing_lancamento", {}).get("infoproduto", {}).get("nome", "infoproduto")
    slug = re.sub(r"[^a-z0-9]+", "-", produto_name.lower()).strip("-") or "infoproduto"
    stamp = datetime.utcnow().strftime("%Y%m%d-%H%M%S")
    outdir = f"outputs/{slug}-{stamp}"
    os.makedirs(outdir, exist_ok=True)


presentation_path = f"{outdir}/presentation.md"
with open(presentation_path, "w", encoding="utf-8") as f:
    f.write(presentation_md)

print(f"✅ Apresentação corrigida e salva com sucesso em: {presentation_path}")

✅ Apresentação corrigida e salva com sucesso em: outputs/mentoria-de-desenvolvimento-inteligente-20250921-201619/presentation.md
